In [16]:
from PIL import Image, ImageChops, ImageEnhance
from keras.models import load_model
from keras.preprocessing import image
import numpy as np

In [17]:
def convert_to_ela_image(path, quality):
    filename = path
    resaved_filename = filename.split('.')[0] + '.resaved.jpg'
    ELA_filename = filename.split('.')[0] + '.ela.png'
    
    im = Image.open(filename).convert('RGB')
    im.save(resaved_filename, 'JPEG', quality=quality)
    resaved_im = Image.open(resaved_filename)
    
    ela_im = ImageChops.difference(im, resaved_im)
    
    extrema = ela_im.getextrema()
    max_diff = max([ex[1] for ex in extrema])
    if max_diff == 0:
        max_diff = 1
    scale = 255.0 / max_diff
    
    ela_im = ImageEnhance.Brightness(ela_im).enhance(scale)
    
    return ela_im

In [18]:
from skimage import transform
def load(filename):
    image = Image.open(filename)
    np_image = convert_to_ela_image(filename, 90)
    np_image = np.array(np_image).astype('float32')/255
    np_image = transform.resize(np_image, (128, 128, 3))
    np_image = np.expand_dims(np_image, axis=0)
    return np_image

In [19]:
model = load_model('saved-models/model-v1-with-casia.h5')

In [31]:
def predict_image(img):
    if bool(model.predict_classes(img)[0]):
        return "Tampered Image"
    return "Real Image"

In [38]:
print('++++ REAL IMAGE ++++')
real = load('custom test data/real-3.jpg')
print(predict_image(real), end="\n\n")

print('++++ FAKE IMAGE ++++')
fake = load('custom test data/fake-3.jpg')
print(predict_image(fake), end="\n\n")

print('++++ REAL IMAGE ++++')
real = load('datasets/dataset-v1/CASIA2/Au/Au_txt_30020.jpg')
print(predict_image(real), end="\n\n")

print('++++ FAKE IMAGE ++++')
fake = load('datasets/dataset-v1/CASIA2/Tp/Tp_D_NNN_M_B_nat10128_ani00058_11541.jpg')
print(predict_image(fake))

++++ REAL IMAGE ++++
Real Image

++++ FAKE IMAGE ++++
Tampered Image

++++ REAL IMAGE ++++
Real Image

++++ FAKE IMAGE ++++
Tampered Image
